In [1]:
import numpy as np
import copy

class BranchAndBound:
    def __init__(self, a_array, c_array,  a_limit):
        self.a_limit = a_limit # 予算
        self.a_array = np.array(a_array) # 経費
        self.c_array = np.array(c_array) #期待利得
        self.r_array = self.c_array / self.a_array # 経費あたりの期待利得
        self.r_index_sorted =  np.argsort(self.r_array)[::-1] # rの値が大きい順にインデックスを並び替えた配列． 探索順． 例：r_array =  [2.0 1.7  2.4  2.3] =>  r_index_sorted = [2 3 0 1] ．
        self.x_array = np.zeros(len(self.r_array)) # 解の初期値
        self.f_max = 0 # 目的関数の初期値
        self.p_list = []  # 部分問題のリスト
        
    # 貪欲法
    def greedy(self):
        a_sum = 0
        x_array = np.zeros(len(self.r_array))
        for index in self.r_index_sorted:
            if a_sum + self.a_array[index] <= self.a_limit:
                x_array[index] = 1
                a_sum += self.a_array[index]
        return (x_array, sum(x_array * self.c_array))
        
    # 部分問題と暫定解が与えられた時に終端条件を満たすか判定する(終端条件を満たすときFalseを返す)，　必要に応じて暫定解の更新も行う．
    def relaxation(self, r_index_fixed, f_max): # r_index_fixed = [[0で固定するxのインデックス], [1で固定するxのインデックス]]
        # 部分問題の条件を満たすxを作る． 自由変数は0で初期化
        x_array = np.zeros(len(self.r_array))
        x_array[[r_index_fixed[0]]] = 0
        x_array[[r_index_fixed[1]]] = 1
        a_sum = sum(x_array * self.a_array)
        c_sum = sum(x_array * self.c_array)
        r_index_free = copy.deepcopy(self.r_index_sorted[len(r_index_fixed[0]) + len(r_index_fixed[1]) -1:]) # 自由変数のインデックス． r_index_sortedに影響を与えないように深いコピーを利用．
        free_len = len(r_index_free) # 自由変数の個数

        if free_len == 0: # 自由変数の個数が0
            if  self.f_max < c_sum: # 暫定解よりも目的関数の値が大きければ暫定解を更新
                self.f_max = c_sum
                self.x_array = x_array.astype(np.int64)
            return False
        elif a_sum > self.a_limit: #実行可能解がない
            return False
        else: # 実行可能解は持つ．
            # 連続緩和問題を解く．
            zero_or_one = True # 0-1条件を満たすか
            for index in r_index_free: 
                if a_sum + self.a_array[index] <= self.a_limit:
                    x_array[index] = 1
                    a_sum  +=  self.a_array[index]
                elif a_sum <= self.a_limit:
                    x_array[index] = (self.a_limit - a_sum) / self.a_array[index]
                    a_sum += self.a_array[index] * x_array[index]
                    if x_array[index] != 0:
                        zero_or_one  = False #0-1条件を満たさない．
                else:
                    x_array[index] = 0
            x_relaxation = sum(x_array * self.c_array) # 連続緩和問題の解
            if (zero_or_one): # 0-1条件を満たす                                   
                if  self.f_max < x_relaxation: # 暫定解よりも目的関数の値が大きければ暫定解を更新
                    self.f_max = x_relaxation
                    self.x_array = x_array.astype(np.int64)
                return False 
            else: 
                return self.f_max < x_relaxation # 緩和問題の解の方が小さい場合は終端条件を満たし探索終了．
                
    # 分岐限定法を利用して解を求める関数
    def branch_and_bound(self):
        # 暫定解を貪欲法の解に設定．
        self.x_array = np.array(self.greedy()[0]).astype(np.int64)
        self.f_max = self.greedy()[1]    
        
        # 部分問題の生成
        self.p_list =[[[self.r_index_sorted[0]],[]], [[], [self.r_index_sorted[0]]]]# 経費あたりの価値が最大のものを場合分けして部分問題を生成 #  [[[2], []], [[], [2]]]
        self.r_index_sorted = np.delete(self.r_index_sorted,0)
        
        while(len(self.p_list) > 0): # 部分問題がなくなるまで繰り返す
            r_index_fixed = self.p_list[0] # 部分問題の先頭を取得
            if (self.relaxation(r_index_fixed, self.f_max)): # 終端条件を満たさない場合(さらに探索が必要な場合)
                # 先頭の部分問題を分割する．
                f1 = copy.deepcopy(self.p_list[0])# 例：[[2],[]]
                f1[0].append(int(self.r_index_sorted[0])) #  [2] => [2,3]
                f1 = [f1[0], f1[1]] # 例：[[2,3],[]]
                f2 = copy.deepcopy(self.p_list[0]) 
                f2[1].append(int(self.r_index_sorted[0]))
                f2 = [f2[0], f2[1]]
                self. p_list.pop(0) # 先頭の部分問題を消去
                self.p_list.insert(0, f1) # 先頭に新たな部分問題を追加(Queue)
                self.p_list.insert(0, f2)
            else: # 終端条件を満たす場合
                self. p_list.pop(0) # 先頭の部分問題を消去
                
        print("解：[x1, x2, x3, x4] = ", list(self.x_array))
        print("最大値：", int(self.f_max))

In [2]:
q = BranchAndBound(a_array = [2, 3, 5, 6], c_array = [4, 5, 12, 14], a_limit = 9)
q.branch_and_bound()

解：[x1, x2, x3, x4] =  [0, 1, 0, 1]
最大値： 19


In [3]:
q.greedy()

(array([ 1.,  0.,  0.,  1.]), 18.0)